# Daily Results

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import sys
import glob

from datetime import datetime
import time

import csv
import json

In [2]:
projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

In [3]:
CONFIG_DIR = 'config'
SESSIONS_DIR = 'sessions'
GPSDATA_DIR = 'gpsdata'

In [4]:
CONFIG_JSON = 'config.json'
ENTRANTS_JSON = 'entrants.json'
SESSIONS_JSON = 'sessions.json'

In [5]:
DEFAULT_NATIONALITY = 'England'

In [6]:
try:
    wsw_full_refresh = int(os.environ['WSW_FULL_REFRESH'])
except:
    wsw_full_refresh = 1

## Generic Class

Generic class to ensure that all custom classes are printable

In [7]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

## Entrant Class

Class to manage entrants

In [8]:
class Entrant(Printable):
    def __init__(self, entrantId, entrantDict, iswcRequired=False):
        '''Initialise entrant object'''

        # Entrant ID is generated by the calling routine
        self.entrantId = entrantId
        
        # Retain original dictionary for later use by reports and filters
        self.entrantDict = entrantDict
        
        # Quickly check if entrant details as defined in the config
        for key in config['Entrants']:
            if key != 'ISWC Member' or iswcRequired:
                # Check the attribute is present
                if key not in entrantDict:
                    raise ValueError('Missing "{}" for entrant #{}'.format(key, entrantId))
                # Check the attribute is valid
                if entrantDict[key] not in config['Entrants'][key]:
                    raise ValueError('Unexpected "{}" for entrant #{} - {}'.format(key, entrantId, entrantDict[key]))

        # Record personal details
        self.firstName = entrantDict['First Name']
        self.familyName = entrantDict['Family Name']
        self.gender = entrantDict['Gender']

        # Record nationality
        try:
            self.nationality = entrantDict['Nationality']
        except:
            self.nationality = DEFAULT_NATIONALITY
            
        # Record fleet details
        self.craftType = entrantDict['Craft Type']
        self.status = entrantDict['Status']
        self.entryType = entrantDict['EntryType']
        self.firstTimer = entrantDict['First Timer']

        # Record membership details
        self.ukwaMember = entrantDict['UKWA Member']
        if iswcRequired:
            self.iswcMember = entrantDict['ISWC Member']

## Course Class

Class to manage courses - start / end times

In [9]:
class Course(Printable):
    def __init__(self, courseId, courseDict):
        '''Initialise course object'''

        self.courseId = courseId
        self.startTime = courseDict['Start Time']
        self.endTime = courseDict['End Time']

## Session Class

Class to manage sessions

In [10]:
class Session(Printable):
    def __init__(self, path):
        '''Initialise session object'''

        self.path = path
        self.date = os.path.basename(path)

        self.courses = {}


    def loadConfig(self):
        '''Read config from JSON'''

        filename = os.path.join(self.path, SESSIONS_JSON)
        with open(filename, 'r', encoding='utf-8') as f:
            jsonTxt = f.read()
            courseDict = json.loads(jsonTxt)
            
            for courseId in courseDict:
                if courseId in self.courses:
                    raise ValueError('Duplicate course "{}" for {}'.format(courseId, self.date))
                course = Course(courseId, courseDict[courseId])
                self.courses[courseId] = course

## Event Class

Class to manage events

In [11]:
class Event():
    def __init__(self, path):
        self.path = path
        self.year = int(os.path.basename(path))

        if self.year in config['Series']['ISWC']['Years']:
            self.iswcYear = True
        else:
            self.iswcYear = False

        self.entrants = {}
        self.gt31Ids = {}
        self.gt31Serials = {}
        
        self.sessions = {}


    def processEvent(self):
        '''Read entrants from JSON'''

        self.loadEntrants()  
        self.indexGt31s()
        self.summariseEntrants()
       
        self.loadSessions()


    def loadEntrants(self):
        '''Read entrants from JSON'''

        filename = os.path.join(self.path, CONFIG_DIR, ENTRANTS_JSON)
        with open(filename, 'r', encoding='utf-8') as f:
            jsonTxt = f.read()
            entrantDicts = json.loads(jsonTxt)
            
            for entrantDict in entrantDicts:
                entrantId = len(self.entrants)
                entrant = Entrant(entrantId, entrantDict, iswcRequired=self.iswcYear)
                self.entrants[entrantId] = entrant


    def indexGt31s(self):
        '''Create GT-31 indices for entrants'''

        for entrantId in self.entrants.keys():
            entrant = self.entrants[entrantId]
            if 'GT31 ID' in entrant.entrantDict:
                self.gt31Ids[entrant.entrantDict['GT31 ID']] = entrant
            if 'GT31 SN' in entrant.entrantDict:
                self.gt31Serials[entrant.entrantDict['GT31 SN']] = entrant
        

    def summariseEntrants(self):
        '''Print summary of entrants'''
        
        print('{} - {} entrants loaded'.format(self.year, len(self.entrants)))
        

    def loadSessions(self):
        '''Load all of the event sessions'''
        
        sessionPaths = sorted(glob.glob(os.path.join(self.path, SESSIONS_DIR, '20[0-9][0-9][0-1][0-9][0-3][0-9]')))
        
        for sessionPath in sessionPaths:
            session = Session(sessionPath)
            session.loadConfig()
            self.sessions[session.date] = session

## Process Years

Process all available years

In [12]:
pc1 = time.perf_counter()

In [13]:
# Read main config
filename = os.path.join(projdir, CONFIG_DIR, CONFIG_JSON)
with open(filename, 'r', encoding='utf-8') as f:
    jsonTxt = f.read()
    config = json.loads(jsonTxt)

In [14]:
# Only process the current year (for now)
eventPaths = sorted(glob.glob(os.path.join(projdir, '20[0-9][0-9]')))
year = datetime.now().year
for eventPath in eventPaths:
    if int(os.path.basename(eventPath)) <= year:
        if int(os.path.basename(eventPath)) == year or wsw_full_refresh:
            event = Event(eventPath)
            event.processEvent()

2018 - 84 entrants loaded
2019 - 68 entrants loaded
2021 - 50 entrants loaded


In [15]:
pc2 = time.perf_counter()
print("Reports completed in %0.2f seconds" % (pc2 - pc1))

Reports completed in 0.07 seconds


## All Done!